In [1]:
import math
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/Eshwar/PlayerStatistics.csv'

df = pd.read_csv(path)

df.head()

Mounted at /content/drive


/tmp/ipython-input-1740041064.py:6: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,James,Johnson,201949,42400407,22-06-2025 20:00,Indiana,Pacers,Oklahoma City,Thunder,Playoffs,...,0.000,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,T.J.,McConnell,204456,42400407,22-06-2025 20:00,Indiana,Pacers,Oklahoma City,Thunder,Playoffs,...,0.000,0.0,0.0,0.000,6.0,0.0,6.0,2.0,7.0,-12.0
2,Myles,Turner,1626167,42400407,22-06-2025 20:00,Indiana,Pacers,Oklahoma City,Thunder,Playoffs,...,0.333,4.0,1.0,0.250,3.0,1.0,4.0,3.0,1.0,-12.0
3,Pascal,Siakam,1627783,42400407,22-06-2025 20:00,Indiana,Pacers,Oklahoma City,Thunder,Playoffs,...,0.400,7.0,4.0,0.571,3.0,1.0,4.0,2.0,1.0,-4.0
4,Alex,Caruso,1627936,42400407,22-06-2025 20:00,Oklahoma City,Thunder,Indiana,Pacers,Playoffs,...,0.400,0.0,0.0,0.000,1.0,2.0,3.0,3.0,0.0,8.0


###Data preprocessing

In [3]:
# Filter for regular season only
df_filtered = df[df['gameType'] == 'Regular Season']

df_filtered = df_filtered[~df_filtered['gameSubLabel'].str.contains('Championship', case=False, na=False)].copy()

df_filtered.head()

,firstName,lastName,personId,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,gameType,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
2572,Jeff,Green,201145,22401193,13-04-2025 15:30,Houston,Rockets,Denver,Nuggets,Regular Season,...,0.200,0.0,0.0,0.000,2.0,1.0,3.0,1.0,0.0,-14.0
2573,Russell,Westbrook,201566,22401193,13-04-2025 15:30,Denver,Nuggets,Houston,Rockets,Regular Season,...,0.000,8.0,7.0,0.875,0.0,0.0,0.0,1.0,1.0,8.0
2574,DeAndre,Jordan,201599,22401193,13-04-2025 15:30,Denver,Nuggets,Houston,Rockets,Regular Season,...,0.000,0.0,0.0,0.000,3.0,2.0,5.0,0.0,0.0,-9.0
2575,Steven,Adams,203500,22401193,13-04-2025 15:30,Houston,Rockets,Denver,Nuggets,Regular Season,...,0.000,0.0,0.0,0.000,4.0,2.0,6.0,2.0,1.0,-23.0
2576,Aaron,Gordon,203932,22401193,13-04-2025 15:30,Denver,Nuggets,Houston,Rockets,Regular Season,...,0.333,3.0,3.0,1.000,3.0,4.0,7.0,0.0,3.0,21.0


In [4]:
df_filtered['gameDate'] = pd.to_datetime(df_filtered['gameDate'])

start_datetime = pd.to_datetime('2010-10-26 00:00:00')
end_datetime = pd.to_datetime('2025-12-31 23:59:59')

print("Before datetime filtering:")
print(f"Total rows: {df_filtered.shape[0]:,}")
print(f"Date range: {df_filtered['gameDate'].min()} to {df_filtered['gameDate'].max()}")

df_final = df_filtered[(df_filtered['gameDate'] >= start_datetime) &
                       (df_filtered['gameDate'] <= end_datetime)].copy()

print(f"\nAfter datetime filtering (2010-10-26 to 2025-12-31):")
print(f"Total rows: {df_final.shape[0]:,}")
print(f"Removed rows: {df_filtered.shape[0] - df_final.shape[0]:,}")
print(f"Exact datetime range: {df_final['gameDate'].min()} to {df_final['gameDate'].max()}")


/tmp/ipython-input-30698688.py:1: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_filtered['gameDate'] = pd.to_datetime(df_filtered['gameDate'])


Before datetime filtering:
Total rows: 925,792
Date range: 1993-02-24 20:00:00 to 2025-04-13 15:30:00

After datetime filtering (2010-10-26 to 2025-12-31):
Total rows: 455,211
Removed rows: 470,581
Exact datetime range: 2010-10-26 19:30:00 to 2025-04-13 15:30:00


In [5]:
# All-Star Game dates from 2011-2025
all_star_dates = [
    ('2011-02-18', '2011-02-20'),  # 2011: February 18–20
    ('2012-02-24', '2012-02-26'),  # 2012: February 24–26
    ('2013-02-15', '2013-02-17'),  # 2013: February 15–17
    ('2014-02-14', '2014-02-16'),  # 2014: February 14–16
    ('2015-02-13', '2015-02-15'),  # 2015: February 13–15
    ('2016-02-12', '2016-02-14'),  # 2016: February 12–14
    ('2017-02-17', '2017-02-19'),  # 2017: February 17–19
    ('2018-02-16', '2018-02-18'),  # 2018: February 16–18
    ('2019-02-15', '2019-02-17'),  # 2019: February 15–17
    ('2020-02-14', '2020-02-16'),  # 2020: February 14–16
    ('2021-03-07', '2021-03-07'),  # 2021: March 7 (COVID delayed)
    ('2022-02-18', '2022-02-20'),  # 2022: February 18–20
    ('2023-02-17', '2023-02-19'),  # 2023: February 17–19
    ('2024-02-16', '2024-02-18'),  # 2024: February 16–18
    ('2025-02-14', '2025-02-16'),  # 2025: February 14–16
]

all_star_ranges = [(pd.to_datetime(start), pd.to_datetime(end))
                   for start, end in all_star_dates]

def is_all_star_date(game_date):
    for start_date, end_date in all_star_ranges:
        if start_date <= game_date <= end_date:
            return True
    return False

# Remove All-Star dates from dataset
print("Before removing All-Star dates:")
print(f"Total games: {len(df_final):,}")

df_clean = df_final[~df_final['gameDate'].apply(is_all_star_date)].copy()

print(f"\nAfter removing All-Star dates:")
print(f"Total games: {len(df_clean):,}")
print(f"Removed games: {len(df_final) - len(df_clean):,}")

removed_games = df_final[df_final['gameDate'].apply(is_all_star_date)]
if len(removed_games) > 0:
    print(f"\nRemoved All-Star dates sample:")
    print(removed_games[['gameDate']].drop_duplicates().sort_values('gameDate').head(10))


Before removing All-Star dates:
Total games: 455,211

After removing All-Star dates:
Total games: 455,211
Removed games: 0


###Season segrergation

In [6]:
df_final['gameDate'] = pd.to_datetime(df_final['gameDate'])

# NBA season ranges (inclusive of start AND end dates)
season_ranges = [
    ('2010-10-26', '2011-04-14', '2010-2011'),
    ('2011-12-25', '2012-04-27', '2011-2012'),
    ('2012-10-30', '2013-04-18', '2012-2013'),
    ('2013-10-29', '2014-04-17', '2013-2014'),
    ('2014-10-28', '2015-04-16', '2014-2015'),
    ('2015-10-27', '2016-04-14', '2015-2016'),
    ('2016-10-25', '2017-04-13', '2016-2017'),
    ('2017-10-17', '2018-04-12', '2017-2018'),
    ('2018-10-16', '2019-04-11', '2018-2019'),
    ('2019-10-22', '2020-08-15', '2019-2020'),  # COVID
    ('2020-12-22', '2021-05-17', '2020-2021'),  # COVID shortened
    ('2021-10-19', '2022-04-11', '2021-2022'),
    ('2022-10-18', '2023-04-10', '2022-2023'),
    ('2023-10-24', '2024-04-15', '2023-2024'),
    ('2024-10-22', '2025-04-14', '2024-2025')
]

season_ranges = [(pd.to_datetime(start), pd.to_datetime(end), season_label)
                 for start, end, season_label in season_ranges]

def assign_nba_season(game_date):
    for start_date, end_date, season_name in season_ranges:
        if start_date <= game_date < end_date:
            return season_name
    return 'Unknown'

df_final['Season'] = df_final['gameDate'].apply(assign_nba_season)

print("Season assignment completed!")
print("\nSeason distribution:")
season_counts = df_final['Season'].value_counts().sort_index()
print(season_counts)

print(f"\nTotal games assigned: {(df_final['Season'] != 'Unknown').sum():,}")
print(f"Unknown games: {(df_final['Season'] == 'Unknown').sum():,}")

Season assignment completed!

Season distribution:
Season
2010-2011    29475
2011-2012    25455
2012-2013    31338
2013-2014    31378
2014-2015    31412
2015-2016    31423
2016-2017    31564
2017-2018    30980
2018-2019    30808
2019-2020    26538
2020-2021    28859
2021-2022    31321
2022-2023    31542
2023-2024    30626
2024-2025    32492
Name: count, dtype: int64

Total games assigned: 455,211
Unknown games: 0


In [7]:
print(f"Before filtering: {len(df_clean):,} rows")

# One-line filtering - removes both NaN and zero values
df_filtered_minutes = df_clean[df_clean['numMinutes'] > 0].copy()

print(f"After filtering: {len(df_filtered_minutes):,} rows")
print(f"Removed: {len(df_clean) - len(df_filtered_minutes):,} rows")

def normalize_minutes_seconds(value):

    minutes = int(value)  # Whole number part
    seconds = int((value - minutes) * 100)  # Decimal part * 100 = seconds
    decimal_minutes = minutes + (seconds / 60)
    return decimal_minutes

# Apply normalization to your ACTUAL DataFrame
df_filtered_minutes['min_dec'] = df_filtered_minutes['numMinutes'].apply(normalize_minutes_seconds)
print(df_filtered_minutes[['numMinutes','min_dec']].head())
df_filtered_minutes['Season'] = df_filtered_minutes['gameDate'].apply(assign_nba_season)
df_filtered_minutes.to_csv('nba_complete_dataset.csv', index=False)


Before filtering: 455,211 rows
After filtering: 372,982 rows
Removed: 82,229 rows
      numMinutes    min_dec
2572       11.31  11.516667
2573       22.39  22.650000
2574       10.52  10.850000
2575       17.13  17.200000
2576       26.23  26.383333


In [8]:
minutes_per_season = df_filtered_minutes.groupby(['personId', 'Season'])['min_dec'].sum().reset_index()

In [9]:
# Filter for players with at least 500 total minutes in the season
qualified_players = minutes_per_season[minutes_per_season['min_dec'] >= 500]

In [10]:
# Seasons starting from 2010-11 onwards
valid_seasons = [f"{year}-{year+1}" for year in range(2010, 2025)]
qualified_players = qualified_players[qualified_players['Season'].isin(valid_seasons)]

In [11]:
df_qualified = df_filtered_minutes.merge(qualified_players[['personId', 'Season']], on=['personId', 'Season'], how='inner')

In [12]:
season_stats = df_qualified.groupby(['personId', 'Season']).agg({
    'gameId': 'nunique',
    'points': 'sum',
    'assists': 'sum',
    'reboundsDefensive': 'sum',
    'reboundsOffensive': 'sum',
    'reboundsTotal': 'sum',
    'steals': 'sum',
    'blocks': 'sum',
    'fieldGoalsMade': 'sum',
    'fieldGoalsAttempted': 'sum',
    'freeThrowsMade': 'sum',
    'freeThrowsAttempted': 'sum',
    'threePointersMade': 'sum',
    'turnovers': 'sum',
    'foulsPersonal': 'sum',
    'min_dec': 'sum',
}).reset_index()

season_stats.rename(columns={'gameId': 'games_played'}, inplace=True)

# Calculate per-game averages
season_stats['PPG'] = season_stats['points'] / season_stats['games_played']
season_stats['APG'] = season_stats['assists'] / season_stats['games_played']
season_stats['RPG'] = season_stats['reboundsTotal'] / season_stats['games_played']
season_stats['SPG'] = season_stats['steals'] / season_stats['games_played']
season_stats['BPG'] = season_stats['blocks'] / season_stats['games_played']

season_stats['FGM_per_game'] = season_stats['fieldGoalsMade'] / season_stats['games_played']
season_stats['FGA_per_game'] = season_stats['fieldGoalsAttempted'] / season_stats['games_played']
season_stats['FTM_per_game'] = season_stats['freeThrowsMade'] / season_stats['games_played']
season_stats['FTA_per_game'] = season_stats['freeThrowsAttempted'] / season_stats['games_played']
season_stats['3PTM_per_game'] = season_stats['threePointersMade'] / season_stats['games_played']

season_stats['TO_per_game'] = season_stats['turnovers'] / season_stats['games_played']
season_stats['PF_per_game'] = season_stats['foulsPersonal'] / season_stats['games_played']


In [13]:
player_names = df_filtered_minutes[['personId', 'firstName', 'lastName']].drop_duplicates(subset='personId')
season_stats = season_stats.merge(player_names, on='personId', how='left')


In [14]:
season_stats.head()

,personId,Season,games_played,points,assists,reboundsDefensive,reboundsOffensive,reboundsTotal,steals,blocks,...,BPG,FGM_per_game,FGA_per_game,FTM_per_game,FTA_per_game,3PTM_per_game,TO_per_game,PF_per_game,firstName,lastName
0,255,2010-2011,80,1056.0,200.0,261.0,77.0,338.0,64.0,35.0,...,0.437500,4.987500,10.312500,2.662500,3.212500,0.562500,1.675000,2.300000,Grant,Hill
1,255,2011-2012,49,499.0,107.0,142.0,29.0,171.0,41.0,29.0,...,0.591837,4.102041,9.204082,1.693878,2.224490,0.285714,1.326531,1.795918,Grant,Hill
2,406,2010-2011,37,341.0,26.0,130.0,48.0,178.0,16.0,42.0,...,1.135135,3.621622,5.432432,1.972973,3.540541,0.000000,1.513514,3.243243,Shaquille,O'Neal
3,436,2010-2011,56,139.0,23.0,86.0,36.0,122.0,10.0,4.0,...,0.071429,0.982143,2.232143,0.517857,0.625000,0.000000,0.446429,1.089286,Juwan,Howard
4,467,2010-2011,80,630.0,655.0,316.0,35.0,351.0,134.0,29.0,...,0.362500,2.687500,7.450000,0.837500,0.962500,1.662500,2.237500,1.487500,Jason,Kidd


In [15]:
# True Shooting Attempts (TSA)
season_stats['TSA'] = season_stats['fieldGoalsAttempted'] + 0.44 * season_stats['freeThrowsAttempted']

# Replace zero TSA with a small epsilon to avoid division by zero
season_stats['TSA'] = season_stats['TSA'].replace(0, 1e-10)

# TS%
season_stats['TS_percent'] = season_stats['points'] / (2 * season_stats['TSA'])

season_stats['TS_percent'] = season_stats['TS_percent'] * 100

print(season_stats[['personId', 'Season', 'points', 'fieldGoalsAttempted', 'freeThrowsAttempted', 'TS_percent']].head())

   personId     Season  points  fieldGoalsAttempted  freeThrowsAttempted  \
0       255  2010-2011  1056.0                825.0                257.0   
1       255  2011-2012   499.0                451.0                109.0   
2       406  2010-2011   341.0                201.0                131.0   
3       436  2010-2011   139.0                125.0                 35.0   
4       467  2010-2011   630.0                596.0                 77.0   

   TS_percent  
0   56.285178  
1   50.004008  
2   65.921745  
3   49.501425  
4   50.009526  


In [16]:
season_stats['Offensive_Contribution'] = (
    season_stats['points'] +
    0.7 * season_stats['assists'] +
    0.7 * season_stats['reboundsTotal'] -
    0.4 * season_stats['turnovers'] -
    0.4 * season_stats['foulsPersonal']
)

season_stats['Defensive_Contribution'] = (
    season_stats['steals'] +
    season_stats['blocks'] +
    0.3 * season_stats['reboundsDefensive']
)

# Calculate simplified WS proxy per 48 minutes
season_stats['WS_Proxy'] = ((season_stats['Offensive_Contribution'] +
                            season_stats['Defensive_Contribution']) /
                            season_stats['min_dec']) * 48


In [17]:
season_stats['FG_missed'] = season_stats['fieldGoalsAttempted'] - season_stats['fieldGoalsMade']
season_stats['FT_missed'] = season_stats['freeThrowsAttempted'] - season_stats['freeThrowsMade']

season_stats['PER_Proxy'] = (
    season_stats['points'] +
    season_stats['reboundsTotal'] +
    season_stats['assists'] +
    2 * season_stats['steals'] +
    2 * season_stats['blocks'] -
    season_stats['FG_missed'] -
    0.7 * season_stats['FT_missed'] -
    season_stats['turnovers']
) / season_stats['min_dec'] * 48

# View top player seasons by PER proxy
print(season_stats[['personId', 'Season', 'PER_Proxy']].sort_values(by='PER_Proxy', ascending=False).head(10))


      personId     Season  PER_Proxy
3328    203954  2023-2024  62.025263
3050    203507  2019-2020  60.478473
3382    203999  2021-2022  60.210492
3385    203999  2024-2025  58.863931
3383    203999  2022-2023  58.283916
3049    203507  2018-2019  57.869082
3384    203999  2023-2024  57.332915
3052    203507  2021-2022  56.845714
2488    203076  2018-2019  55.882133
3054    203507  2023-2024  55.629386


In [18]:
season_stats = season_stats.drop_duplicates(subset=['personId', 'Season'])
print(season_stats.groupby(['personId', 'Season']).size().value_counts())


1    5250
Name: count, dtype: int64


In [19]:
season_stats.head()

,personId,Season,games_played,points,assists,reboundsDefensive,reboundsOffensive,reboundsTotal,steals,blocks,...,firstName,lastName,TSA,TS_percent,Offensive_Contribution,Defensive_Contribution,WS_Proxy,FG_missed,FT_missed,PER_Proxy
0,255,2010-2011,80,1056.0,200.0,261.0,77.0,338.0,64.0,35.0,...,Grant,Hill,938.08,56.285178,1305.4,177.3,30.067427,426.0,44.0,24.358935
1,255,2011-2012,49,499.0,107.0,142.0,29.0,171.0,41.0,29.0,...,Grant,Hill,498.96,50.004008,632.4,112.6,26.430155,250.0,26.0,20.711308
2,406,2010-2011,37,341.0,26.0,130.0,48.0,178.0,16.0,42.0,...,Shaquille,O'Neal,258.64,65.921745,413.4,97.0,33.377657,67.0,58.0,32.527520
3,436,2010-2011,56,139.0,23.0,86.0,36.0,122.0,10.0,4.0,...,Juwan,Howard,140.40,49.501425,206.1,39.8,20.927660,70.0,6.0,18.110638
4,467,2010-2011,80,630.0,655.0,316.0,35.0,351.0,134.0,29.0,...,Jason,Kidd,629.88,50.009526,1215.0,257.8,27.075603,381.0,10.0,25.645347


In [20]:
from sklearn.preprocessing import MinMaxScaler

# Metrics to normalize
metrics = ['PPG', 'RPG', 'APG', 'SPG', 'BPG', 'TS_percent', 'WS_Proxy', 'PER_Proxy']


scaler = MinMaxScaler()


season_stats_scaled = scaler.fit_transform(season_stats[metrics])

# Create DataFrame of scaled metrics with same index
scaled_df = pd.DataFrame(season_stats_scaled, columns=metrics, index=season_stats.index)

# Assign weights to each metric
weights = {
    'PPG': 0.15,
    'RPG': 0.12,
    'APG': 0.12,
    'SPG': 0.10,
    'BPG': 0.10,
    'TS_percent': 0.15,
    'WS_Proxy': 0.13,
    'PER_Proxy': 0.13
}

# Weighted sum composite score
season_stats['composite_score'] = np.zeros(len(season_stats))
for metric, weight in weights.items():
    season_stats['composite_score'] += scaled_df[metric] * weight

# Rank player seasons by composite score descending
season_stats = season_stats.sort_values(by='composite_score', ascending=False).reset_index(drop=True)

# Extract rankings
top_10 = season_stats.loc[:9, ['personId','firstName','lastName', 'Season', 'composite_score']]
mid_start = len(season_stats) // 2 - 5
mid_10 = season_stats.loc[mid_start:mid_start+9, ['personId','firstName','lastName','Season', 'composite_score']]
bottom_10 = season_stats.tail(10)[['personId','firstName','lastName', 'Season', 'composite_score']]


print("10 Best Player Seasons:")
print(top_10)

print("\n10 Most Average Player Seasons:")
print(mid_10)

print("\n10 Worst Player Seasons:")
print(bottom_10)

10 Best Player Seasons:
   personId firstName       lastName     Season  composite_score
0    203999    Nikola          Jokic  2024-2025         0.737270
1    203954      Joel         Embiid  2023-2024         0.721877
2    203999    Nikola          Jokic  2021-2022         0.711196
3    203999    Nikola          Jokic  2022-2023         0.692373
4    203507   Giannis  Antetokounmpo  2019-2020         0.683279
5    203999    Nikola          Jokic  2023-2024         0.681892
6    203507   Giannis  Antetokounmpo  2018-2019         0.680271
7   1629029      Luka         Doncic  2023-2024         0.676306
8    203507   Giannis  Antetokounmpo  2023-2024         0.670970
9    203507   Giannis  Antetokounmpo  2021-2022         0.666954

10 Most Average Player Seasons:
      personId firstName    lastName     Season  composite_score
2620   1630692    Jordan     Goodwin  2022-2023         0.269768
2621    201155    Rodney     Stuckey  2014-2015         0.269759
2622   1629673    Jordan       Po